In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

def generate_time_series(batch_size, n_steps):
  freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
  time = np.linspace(0, 1, n_steps)
  series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))
  series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20))
  series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
  return series[..., np.newaxis].astype(np.float32)

n_steps = 50
series = generate_time_series(10000, n_steps + 10)
X_train = series[:7000, :n_steps]
X_valid = series[7000:9000, :n_steps]
X_test = series[9000:, :n_steps]
Y = np.empty((10000, n_steps, 10))
for step_ahead in range(1, 10 + 1):
    Y[..., step_ahead - 1] = series[..., step_ahead:step_ahead + n_steps, 0]
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]

In [4]:
def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[None, 1]))
for rate in (1, 2, 4, 8) * 2:
  model.add(keras.layers.Conv1D(filters=20, kernel_size=2, padding="causal",
                                activation="relu", dilation_rate=rate))
model.add(keras.layers.Conv1D(filters=10, kernel_size=1))
model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=20, 
                    validation_data=(X_valid, Y_valid))

Epoch 1/20
219/219 [==============================] - 3s 15ms/step - loss: 0.0657 - last_time_step_mse: 0.0533 - val_loss: 0.0355 - val_last_time_step_mse: 0.0221
Epoch 2/20
219/219 [==============================] - 3s 14ms/step - loss: 0.0314 - last_time_step_mse: 0.0184 - val_loss: 0.0292 - val_last_time_step_mse: 0.0166
Epoch 3/20
219/219 [==============================] - 3s 14ms/step - loss: 0.0274 - last_time_step_mse: 0.0150 - val_loss: 0.0263 - val_last_time_step_mse: 0.0135
Epoch 4/20
219/219 [==============================] - 3s 14ms/step - loss: 0.0254 - last_time_step_mse: 0.0133 - val_loss: 0.0253 - val_last_time_step_mse: 0.0132
Epoch 5/20
219/219 [==============================] - 3s 14ms/step - loss: 0.0241 - last_time_step_mse: 0.0120 - val_loss: 0.0237 - val_last_time_step_mse: 0.0113
Epoch 6/20
219/219 [==============================] - 3s 14ms/step - loss: 0.0232 - last_time_step_mse: 0.0113 - val_loss: 0.0231 - val_last_time_step_mse: 0.0114
Epoch 7/20
219/219 [==